In [38]:
# 모듈 로딩
import torch    # 텐서 관련 모듈
import torch.nn as nn  # 인공신경망 관련 모듈
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F  # 인공신경망 관련 함수들 모듈 (손실 함수, 활성화 함수 등)

# 성능 측정 관련 모듈
from torchmetrics.classification import F1Score  # F1 점수 관련 모듈
from torchmetrics.regression import *  # 회귀 성능 관련
from torchmetrics.classification import *  # 분류 성능 관련

# 모델 정보 출력 모듈
from torchinfo import summary  # 모델 구조 및 정보 관련 모듈

# 데이터 처리 및 시각화 모듈
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [39]:
from urllib.request import urlretrieve
import pandas as pd


DATA_FILE = r'C:\Users\zizonkjs\머신러닝,딥러닝\data\iris.csv'
irisdf=pd.read_csv(DATA_FILE)
irisdf


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [40]:
from sklearn.preprocessing import LabelEncoder

class IrisDataset(Dataset):
    def __init__(self, csv_file):
        # 데이터 로딩
        self.data = pd.read_csv(csv_file)
        # 특성과 레이블 분리
        self.X = self.data.iloc[:, :-1].values.astype('float32')  # 특성
        self.y = LabelEncoder().fit_transform(self.data.iloc[:, -1])  # 레이블 (문자열을 숫자로 변환)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.X[idx])
        label = torch.tensor(self.y[idx], dtype=torch.long)
        return features, label


In [41]:
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 64)  # 입력 특성 4개 (IRIS 데이터)
        self.fc2 = nn.Linear(64, 3)  # 3개의 출력 클래스 (Setosa, Versicolor, Virginica)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [42]:
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    return test_loss, accuracy


In [43]:
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # 기울기 초기화
        output = model(data)  # 모델에 데이터 전달
        loss = criterion(output, target)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

        running_loss += loss.item()
    return running_loss / len(train_loader)


In [44]:
import torch

# device 설정 (GPU 사용 가능하면 'cuda', 아니면 'cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# IRIS 데이터셋 로드
iris_dataset = IrisDataset(DATA_FILE)
iris_loader = DataLoader(iris_dataset, batch_size=16, shuffle=True)

# IRIS 모델 설정
iris_model = IrisModel().to(device)  # 모델을 device로 전송 (GPU 또는 CPU)
optimizer = optim.Adam(iris_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # 손실 함수 설정 (예: 교차 엔트로피)

# 학습 및 테스트 실행
for epoch in range(10):
    train_loss = train_model(iris_model, iris_loader, optimizer, criterion, device)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}')


Epoch 1, Train Loss: 1.0712005734443664
Epoch 2, Train Loss: 0.9470101118087768
Epoch 3, Train Loss: 0.8523678243160248
Epoch 4, Train Loss: 0.774714571237564
Epoch 5, Train Loss: 0.7078666865825654
Epoch 6, Train Loss: 0.6664434850215912
Epoch 7, Train Loss: 0.6018160045146942
Epoch 8, Train Loss: 0.5792315810918808
Epoch 9, Train Loss: 0.5430630147457123
Epoch 10, Train Loss: 0.5118556559085846
